In [3]:
import tensorflow as tf
import math
from IPython import display
import numpy as np
import pandas as pd
from sklearn import metrics
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from tensorflow.python.data import Dataset
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
print("Completed")

Completed


In [6]:
california_housing_dataframe = pd.read_csv('/Users/biplabkumardas/google-tensorflow/california_housing_train.csv', sep=",")
print("Completed")

Completed


In [7]:
california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
10499,-120.4,34.9,8.0,3119.0,620.0,1159.0,544.0,3.5,165.5
14517,-122.2,38.1,10.0,3008.0,532.0,1381.0,522.0,5.4,195.8
9386,-119.2,34.2,22.0,6465.0,1397.0,2694.0,1370.0,3.0,165.6
3426,-117.9,33.9,20.0,3349.0,685.0,1822.0,675.0,4.7,227.0
16223,-122.5,37.8,48.0,2660.0,616.0,1491.0,602.0,4.0,348.6
...,...,...,...,...,...,...,...,...,...
790,-117.1,32.6,25.0,1268.0,282.0,991.0,299.0,3.0,123.6
1444,-117.2,33.9,3.0,21060.0,3366.0,9623.0,2812.0,4.2,143.0
13870,-122.0,37.3,34.0,1764.0,243.0,692.0,223.0,8.0,500.0
4507,-118.0,33.9,24.0,2233.0,347.0,1162.0,355.0,5.6,279.2


In [8]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0
